In [136]:
header = ['color', 'diameter', 'label']

In [137]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

<hr>

In [138]:
def unique_vals(rows, col):
    '''Возвращаем уникальные значения столбцов в датасете'''
    return set([row[col] for row in rows])

In [139]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

<hr>

In [140]:
def class_counts(rows):
    '''Возвращает колличество типов\классов находящихся в датасете'''
    counts = {}
    for row in rows:
        label  = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [141]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

<hr>

In [142]:
def is_numeric(value):
    """Проверяет является ли значение числом."""
    return isinstance(value, int) or isinstance(value, float)

In [143]:
is_numeric(5)

True

<hr>

In [144]:
class Question:
    
    
    '''
    Вопрос используется для разделения набора данных
    '''
    
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        '''
        Сравнивает значения в примере со значением в вопросе
        '''
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        '''
        Отображает объект в виде строки
        '''
        condition = '=='
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [145]:
Question(1,3)

Is diameter >= 3?

In [146]:
q = Question(0, 'Green')
q

Is color == Green?

In [147]:
# Берём первую строку
example = training_data[0]
print(example)
# должен вернуть True
q.match(example)

['Green', 3, 'Apple']


True

In [148]:
example = training_data[1]
print(example)
# должен вернуть False
q.match(example)

['Yellow', 3, 'Apple']


False

<hr>

In [149]:
def partition(rows, question):
    '''
    разделяет множества\выборку на истенные и ложные подмножества
    '''
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [150]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [151]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

<hr>

In [152]:
def gini(rows):
    '''
    Колличественная мера неопределённости
    Примесь набора данных
    '''
    counts = class_counts(rows)
    print(counts)
    impurity = 1 #100%
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -=prob_of_lbl**2
    print("Процент примеси : %s" % (str(impurity)))
    return impurity

In [153]:
no_mixing = [['Apple'], ['Apple']]
gini(no_mixing)

{'Apple': 2}
Процент примеси : 0.0


0.0

In [154]:
some_mixing = [['Apple'], ['Lemon']]
gini(some_mixing)

{'Apple': 1, 'Lemon': 1}
Процент примеси : 0.5


0.5

In [155]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]

gini(lots_of_mixing)

{'Apple': 1, 'Orange': 1, 'Grape': 1, 'Grapefruit': 1, 'Blueberry': 1}
Процент примеси : 0.7999999999999998


0.7999999999999998

In [156]:
lot_of_mixing = [
    ['Apple'],
    ['Apple'],
    ['Grape'],
    ['Grape'],
    ['Lemon']
]
gini(lot_of_mixing)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}
Процент примеси : 0.6399999999999999


0.6399999999999999

<hr>

In [157]:
def info_gain(left, right, current_uncertainty):
    '''
    Позволит нам найти вопрос, который уменьшает нашу неопределённость больше всего
    Сре́днее арифмети́ческое взве́шенное
    '''
    p = float(len(left)) / (len(left) + len(right))
    print(p)
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [158]:
current_uncertainty = gini(training_data)
current_uncertainty

{'Apple': 2, 'Grape': 2, 'Lemon': 1}
Процент примеси : 0.6399999999999999


0.6399999999999999

In [159]:
# Сколько информации мы получим, разделившись на «Зеленый»?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.2
{'Apple': 1}
Процент примеси : 0.0
{'Apple': 1, 'Grape': 2, 'Lemon': 1}
Процент примеси : 0.625


0.1399999999999999

In [160]:
# Как насчет того, если бы мы разделились на «Красном»?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.4
{'Grape': 2}
Процент примеси : 0.0
{'Apple': 2, 'Lemon': 1}
Процент примеси : 0.4444444444444445


0.37333333333333324

<hr>

In [161]:
def find_best_split(rows):
    '''
    Разобрать!
    '''
    
    """---Инициализируем начальные параметры---"""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # получаем индексы колон
    """----------------------------------------"""
    
    
    for col in range(n_features): # проходим по списку индексов колон
        
        values = set([row[col] for row in rows]) # формируем множество уникальных значений из признаков(колон) 
        
        for val in values: # проходим по значениям признаков
            
            question = Question(col, val) # формируем вопрос
            
            true_rows, false_rows = partition(rows, question) # по вопросу разделяем датасет на достоверный и ложные множества 
            
            if len(true_rows) == 0 or len(false_rows) == 0: # если хоть одно из множеств пусто пропускаем интерацию
                continue
                
            gain = info_gain(true_rows, false_rows, current_uncertainty) # просчитываем inforamtion gain
            
            if gain >= best_gain: 
                best_gain, best_question = gain, question
                
    return best_gain, best_question
                
            
            
            

In [162]:
best_gain, best_question = find_best_split(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}
Процент примеси : 0.6399999999999999
0.2
{'Apple': 1}
Процент примеси : 0.0
{'Apple': 1, 'Grape': 2, 'Lemon': 1}
Процент примеси : 0.625
0.4
{'Grape': 2}
Процент примеси : 0.0
{'Apple': 2, 'Lemon': 1}
Процент примеси : 0.4444444444444445
0.4
{'Apple': 1, 'Lemon': 1}
Процент примеси : 0.5
{'Apple': 1, 'Grape': 2}
Процент примеси : 0.4444444444444444
0.6
{'Apple': 2, 'Lemon': 1}
Процент примеси : 0.4444444444444445
{'Grape': 2}
Процент примеси : 0.0


In [163]:
best_gain

0.37333333333333324

In [164]:
best_question

Is diameter >= 3?

In [165]:
class Leaf:
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [166]:
class Decision_Node:
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [167]:
def build_tree(rows):
    
    gain, quesion = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows,question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    return Decision_Node(quesiton, true_branch, false_branch)